In [1]:
import numpy as np
import pandas as pd

from etl import load_monsters, _mechanics, get_monster_df

In [2]:
monsters, ogl = load_monsters()

In [3]:
monster_df = get_monster_df(monsters)

In [4]:
monster_df[_mechanics]

,challenge_rating,armor_class,hit_dice,hit_points,condition_immunities,damage_immunities,damage_resistances,damage_vulnerabilities,actions,reactions,legendary_actions,special_abilities,size,speed,senses
name,,,,,,,,,,,,,,,
Aboleth,10.00,17,18d10,135,,,,,"[{'name': 'Multiattack', 'desc': 'The aboleth ...",NaN,"[{'name': 'Detect', 'desc': 'The aboleth makes...","[{'name': 'Amphibious', 'desc': 'The aboleth c...",Large,"10 ft., swim 40 ft.","darkvision 120 ft., passive Perception 20"
Acolyte,0.25,10,2d8,9,,,,,"[{'name': 'Club', 'desc': 'Melee Weapon Attack...",NaN,NaN,"[{'name': 'Spellcasting', 'desc': 'The acolyte...",Medium,30 ft.,passive Perception 12
Adult Black Dragon,14.00,19,17d12,195,,acid,,,"[{'name': 'Multiattack', 'desc': 'The dragon c...",NaN,"[{'name': 'Detect', 'desc': 'The dragon makes ...","[{'name': 'Amphibious', 'desc': 'The dragon ca...",Huge,"40 ft., fly 80 ft., swim 40 ft.","blindsight 60 ft., darkvision 120 ft., passive..."
Adult Blue Dracolich,17.00,19,18d12,225,"charmed, exhaustion, frightened, paralyzed, po...","lightning, poison",necrotic,,"[{'name': 'Multiattack', 'desc': 'The dracolic...",NaN,"[{'name': 'Detect', 'desc': 'The dracolich mak...","[{'name': 'Legendary Resistance (3/Day)', 'des...",Huge,"40 ft., burrow 30 ft., fly 80 ft.","blindsight 60 ft., darkvision 120 ft., passive..."
Adult Blue Dragon,16.00,19,18d12,225,,lightning,,,"[{'name': 'Multiattack', 'desc': 'The dragon c...",NaN,"[{'name': 'Detect', 'desc': 'The dragon makes ...","[{'name': 'Legendary Resistance (3/Day)', 'des...",Huge,"40 ft., burrow 30 ft., fly 80 ft.","blindsight 60 ft., darkvision 120 ft., passive..."
Adult Brass Dragon,13.00,18,15d12,172,,fire,,,"[{'name': 'Multiattack', 'desc': 'The dragon c...",NaN,NaN,"[{'name': 'Legendary Resistance (3/Day)', 'des...",Huge,"40 ft., burrow 40 ft., fly 80 ft.","blindsight 60 ft., darkvision 120 ft., passive..."
Adult Bronze Dragon,15.00,19,17d12,212,,lightning,,,"[{'name': 'Multiattack', 'desc': 'The dragon c...",NaN,"[{'name': 'Detect', 'desc': 'The dragon makes ...","[{'name': 'Amphibious', 'desc': 'The dragon ca...",Huge,"40 ft., fly 80 ft., swim 40 ft.","blindsight 60 ft., darkvision 120 ft., passive..."
Adult Copper Dragon,14.00,18,16d12,184,,acid,,,"[{'name': 'Multiattack', 'desc': 'The dragon c...",NaN,"[{'name': 'Detect', 'desc': 'The dragon makes ...","[{'name': 'Legendary Resistance (3/Day)', 'des...",Huge,"40 ft., climb 40 ft., fly 80 ft.","blindsight 60 ft., darkvision 120 ft., passive..."
Adult Gold Dragon,17.00,19,19d12,256,,fire,,,"[{'name': 'Multiattack', 'desc': 'The dragon c...",NaN,"[{'name': 'Detect', 'desc': 'The dragon makes ...","[{'name': 'Amphibious', 'desc': 'The dragon ca...",Huge,"40 ft., fly 80 ft., swim 40 ft.","blindsight 60 ft., darkvision 120 ft., passive..."


In [5]:
def replace_nan(df, column, func):
    for x in df.loc[df[column].isnull(), column].index:
        df.at[x, column] = func()

replace_nan(monster_df, 'reactions', list)
replace_nan(monster_df, 'legendary_actions', list)
replace_nan(monster_df, 'special_abilities', list)

In [6]:
monster_df.actions[0]

[{'attack_bonus': 0,
  'desc': 'The aboleth makes three tentacle attacks.',
  'name': 'Multiattack'},
 {'attack_bonus': 9,
  'damage_bonus': 5,
  'damage_dice': '2d6',
  'desc': "Melee Weapon Attack: +9 to hit, reach 10 ft., one target. Hit: 12 (2d6 + 5) bludgeoning damage. If the target is a creature, it must succeed on a DC 14 Constitution saving throw or become diseased. The disease has no effect for 1 minute and can be removed by any magic that cures disease. After 1 minute, the diseased creature's skin becomes translucent and slimy, the creature can't regain hit points unless it is underwater, and the disease can be removed only by heal or another disease-curing spell of 6th level or higher. When the creature is outside a body of water, it takes 6 (1d12) acid damage every 10 minutes unless moisture is applied to the skin before 10 minutes have passed.",
  'name': 'Tentacle'},
 {'attack_bonus': 9,
  'damage_bonus': 5,
  'damage_dice': '3d6',
  'desc': 'Melee Weapon Attack: +9 to hi

Need to use some intelligence in extracting keywords.

In [7]:
import re

pattern = re.compile(r'\bDC (\d+) (Strength|Dexterity|Constitution|Intelligence|Wisdom|Charisma)\b')
def get_dc(text):
    g = re.search(pattern, text)
    if g:
        return g.groups()
    return []


In [8]:
get_dc(monster_df.actions[0][3]['desc'])

('14', 'Wisdom')

In [9]:
monster_df.reactions[monster_df['reactions'].apply(lambda x: len(x) > 0)]

name
Bandit Captain     [{'name': 'Parry', 'desc': 'The captain adds 2...
Black Pudding      [{'name': 'Split', 'desc': 'When a pudding tha...
Chain Devil        [{'name': 'Unnerving Mask', 'desc': 'When a cr...
Erinyes            [{'name': 'Parry', 'desc': 'The erinyes adds 4...
Gladiator          [{'name': 'Parry', 'desc': 'The gladiator adds...
Knight             [{'name': 'Parry', 'desc': 'The knight adds 2 ...
Marilith           [{'name': 'Parry', 'desc': 'The marilith adds ...
Noble              [{'name': 'Parry', 'desc': 'The noble adds 2 t...
Ochre Jelly        [{'name': 'Split', 'desc': 'When a jelly that ...
Shield Guardian    [{'name': 'Shield', 'desc': 'When a creature m...
Stone Giant        [{'name': 'Rock Catching', 'desc': 'If a rock ...
Name: reactions, dtype: object

In [10]:
from functools import partial, reduce

def string_to_list(x, sep=','):
    return [y.strip() for y in x.split(sep)]


def concat(lists):
    return reduce(lambda x, y: x + y, lists, [])


def get_unique_values(series, sep=','):
    lists = series.apply(partial(string_to_list, sep=sep))
    values = concat(lists)
    return sorted(v for v in set(values) if v)

In [11]:
condition_immunities = get_unique_values(monster_df.condition_immunities)
damage_immunities = get_unique_values(monster_df.damage_immunities, sep=';')
damage_resistances = get_unique_values(monster_df.damage_resistances, sep=';')
damage_vulnerabilities = get_unique_values(monster_df.damage_vulnerabilities)